# Scraping Data Science Job Postings from Indeed




The list of Data Science jobs can be found by simple search on Indeed's main page:
<img src="img/Indeed_All.png",width=500, height=400>


As shown on the bottom right corner, there are 14,245 jobs meeting our search 
criteria at the moment.


Hovewer, Indeed only allows users to see up to 1000 results, or 100 pages with
10 jobs per page:
<img src="img/Indeed_Max_Results.png",width=200, height=400>
After that, the site starts showing some random results. It makes sense if you're looking for a job - most of us wouldn't be able to check over 100 pages. But it complicates things a bit if we're trying to scrape the data


Therefore, we have to be more specific in our queries, and divide the result into smaller chunks in order to collect more postings. For example, search by city names:
<img src="img/Search_by_City.png",width=500, height=400>


To do that, we have to prepare a list of cities to iterate through.

In [1]:
from bs4 import BeautifulSoup
import urllib.request as ur

#### Preparing the list of cities

In [2]:

# The page that contains a list of states
states_url = "https://www.indeed.com/find-jobs.jsp"

# Base url to iterate through state pages
base_url = "https://www.indeed.com/"

# Creating an empty list to collect out results
all_cities = []


# Getting a list of state urls from the state page

html_doc = ur.urlopen(states_url).read()
soup = BeautifulSoup(html_doc, 'html.parser')
states_list = soup.find_all(id="states")

for states in states_list:
        links = states.findAll('a')
        
        for a in links:
            
            if a.text != '':
                state_url = base_url + a['href']

                # Getting a list of cities from each state url
                
                html_doc = ur.urlopen(state_url).read()
                soup = BeautifulSoup(html_doc, 'html.parser')
                
                cities_list = soup.find_all(class_='city')
                
                for cities in cities_list:
                    links = cities.findAll('a')
                    for a in links:
                        
                        try:
                            city_state =  a['href']
                            city_state = city_state.lstrip('/l-').split(',')
                            
                            city = city_state[0].replace('-', '+')
                            state = city_state[1].split('-')[1]
                            
                            # Preparind an adress in required format
                            # and adding it to the list
                            
                            adr = 'l=' + city + '%2C+' + state
                            
                            all_cities.append(adr)
                        except:
                            continue;
                                
        

In [3]:
# There are 1492 locations
len(all_cities)


1492

#### Scraping the postings

In [18]:

# Base link to data science jobs
base_url = "https://www.indeed.com/jobs?q=data+science&"

# Creating an empty frame to collect the results
#frame = []


for i in range(len(all_cities)):
    
    # Going to each city page
    try:
        
        # Setting up the search radius to 10 miles for better coverage
        
        city_url = base_url + all_cities[i] + "&radius=10" 
        print(city_url)
    
        response = ur.urlopen(city_url)
        
        html_doc = response.read()
        
        soup = BeautifulSoup(html_doc, 'html.parser')

        # Chenking the number of search results
    
        total_results  = soup.find(id="searchCount").get_text()
        number_of_jobs = (total_results[total_results.index("of")+2: 
                                        total_results.index("jobs")].strip())
        
        try: 
            last_job_number = int(number_of_jobs)
        except:
            last_job_number = int(number_of_jobs[:number_of_jobs.index(",")]+
                                  number_of_jobs[number_of_jobs.index(",")+1:])
        #print(last_job_number)
    except:
        continue;
    
    # In case when a search returns no results, we get automatically redirected 
    # to the general 'data science' search results page. To avoid that, we check
    # the last job number - if it's over 10000, we were redirected. 
    
    if last_job_number < 10000:
        
        # There are ususally 10 search results per page, unless it's specified 
        # otherwise + 5 promoted jobads that we're not taking into account
        
        jobs_per_page = 10
        
        last_job_number = min(last_job_number, 1000)
        
        print(last_job_number)

        for pgno in range(0, last_job_number, jobs_per_page):
            
            # The link to the first page of the search results looks differently 
            # than the following ones, we have to consider that in our script 
            
            if pgno == 0:
                try:
                    response = ur.urlopen(city_url)
                    html_doc = response.read()    
                except:
                    continue;
                soup = BeautifulSoup(html_doc, 'html.parser')

            else:
                try:
                    response = ur.urlopen(city_url + '&start=' +str(pgno))
                    html_doc = response.read()
                except:
                    continue;
                soup = BeautifulSoup(html_doc, 'html.parser')
                print('next 10 from #', pgno)
                
                
            # Getting the job details one by one
            
            for job in soup.find_all(class_='result'):
                link = job.find(class_="turnstileLink")

                try:
                    jt = link.get('title')
                except:
                    jt = ""

                try:
                    comp = job.find(class_='company').get_text().strip()
                except:
                    comp = ""

                try:
                    date = job.find(class_='date').get_text().strip()
                except:
                    date = ""

                try:
                    loc = job.find(class_='location').get_text().strip()
                except:
                    loc = all_cities[i]
                
                # Search results only return short job descriptions. 
                # We have to go to the job page to get the full job ad
                
                job_url = "http://www.indeed.com" + link.get('href')
                
                try:
                    html_doc = ur.urlopen(job_url).read().decode('utf-8')
                except:
                    html_doc = ""
                    #continue;
                    
                soup = BeautifulSoup(html_doc, 'html.parser')

                try:
                    job_ad = soup.find(class_ ='jobsearch-JobComponent-description').get_text().strip()
                except:
                    job_ad = ""

                try:
                    salary = soup.find(class_ ='jobsearch-JobMetadataHeader-item').get_text().strip()
                except:
                    salary = ""

                frame.append({'job_title': jt, 'job_url': job_url, 'company':comp, 'date': date, 
                              'location':loc, 
                              'salary': salary, 
                              'job_description': job_ad})



https://www.indeed.com/jobs?q=data+science&l=Woodbridge%2C+VA&radius=10
112
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
next 10 from # 50
next 10 from # 60
next 10 from # 70
next 10 from # 80
next 10 from # 90
next 10 from # 100
next 10 from # 110
https://www.indeed.com/jobs?q=data+science&l=Glen+Allen%2C+VA&radius=10
54
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
next 10 from # 50
https://www.indeed.com/jobs?q=data+science&l=Fairfax%2C+VA&radius=10
526
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
next 10 from # 50
next 10 from # 60
next 10 from # 70
next 10 from # 80
next 10 from # 90
next 10 from # 100
next 10 from # 110
next 10 from # 120
next 10 from # 130
next 10 from # 140
next 10 from # 150
next 10 from # 160
next 10 from # 170
next 10 from # 180
next 10 from # 190
next 10 from # 200
next 10 from # 210
next 10 from # 220
next 10 from # 230
next 10 from # 240
next 10 from # 250
next 10 from # 260
n

https://www.indeed.com/jobs?q=data+science&l=Kansas+City%2C+MO&radius=10
35
next 10 from # 10
next 10 from # 20
next 10 from # 30
https://www.indeed.com/jobs?q=data+science&l=Joplin%2C+MO&radius=10
https://www.indeed.com/jobs?q=data+science&l=Fenton%2C+MO&radius=10
7
https://www.indeed.com/jobs?q=data+science&l=Wentzville%2C+MO&radius=10
5
https://www.indeed.com/jobs?q=data+science&l=Arnold%2C+MO&radius=10
5
https://www.indeed.com/jobs?q=data+science&l=Springfield%2C+MO&radius=10
2
https://www.indeed.com/jobs?q=data+science&l=Saint+Peters%2C+MO&radius=10
9
https://www.indeed.com/jobs?q=data+science&l=Florissant%2C+MO&radius=10
8
https://www.indeed.com/jobs?q=data+science&l=Bridgeton%2C+MO&radius=10
20
next 10 from # 10
https://www.indeed.com/jobs?q=data+science&l=Sedalia%2C+MO&radius=10
https://www.indeed.com/jobs?q=data+science&l=Columbia%2C+MO&radius=10
6
https://www.indeed.com/jobs?q=data+science&l=Saint+Joseph%2C+MO&radius=10
https://www.indeed.com/jobs?q=data+science&l=Cape+Girard

36
next 10 from # 10
next 10 from # 20
next 10 from # 30
https://www.indeed.com/jobs?q=data+science&l=Prescott%2C+AZ&radius=10
https://www.indeed.com/jobs?q=data+science&l=Kingman%2C+AZ&radius=10
https://www.indeed.com/jobs?q=data+science&l=Fort+Huachuca%2C+AZ&radius=10
6
https://www.indeed.com/jobs?q=data+science&l=Scottsdale%2C+AZ&radius=10
44
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
https://www.indeed.com/jobs?q=data+science&l=Peoria%2C+AZ&radius=10
1
https://www.indeed.com/jobs?q=data+science&l=Avondale%2C+AZ&radius=10
90
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
next 10 from # 50
next 10 from # 60
next 10 from # 70
next 10 from # 80
https://www.indeed.com/jobs?q=data+science&l=Sedona%2C+AZ&radius=10
https://www.indeed.com/jobs?q=data+science&l=Marana%2C+AZ&radius=10
9
https://www.indeed.com/jobs?q=data+science&l=Tempe%2C+AZ&radius=10
131
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
next 10 from

https://www.indeed.com/jobs?q=data+science&l=Oxford%2C+MS&radius=10
https://www.indeed.com/jobs?q=data+science&l=Pearl%2C+MS&radius=10
1
https://www.indeed.com/jobs?q=data+science&l=Mccomb%2C+MS&radius=10
https://www.indeed.com/jobs?q=data+science&l=Brookhaven%2C+MS&radius=10
https://www.indeed.com/jobs?q=data+science&l=Biloxi%2C+MS&radius=10
https://www.indeed.com/jobs?q=data+science&l=Columbus%2C+MS&radius=10
https://www.indeed.com/jobs?q=data+science&l=Greenville%2C+MS&radius=10
https://www.indeed.com/jobs?q=data+science&l=Corinth%2C+MS&radius=10
https://www.indeed.com/jobs?q=data+science&l=Picayune%2C+MS&radius=10
https://www.indeed.com/jobs?q=data+science&l=Oklahoma+City%2C+OK&radius=10
11
next 10 from # 10
https://www.indeed.com/jobs?q=data+science&l=Stillwater%2C+OK&radius=10
https://www.indeed.com/jobs?q=data+science&l=Yukon%2C+OK&radius=10
12
next 10 from # 10
https://www.indeed.com/jobs?q=data+science&l=Ada%2C+OK&radius=10
https://www.indeed.com/jobs?q=data+science&l=Sand+Spr

next 10 from # 100
next 10 from # 110
next 10 from # 120
next 10 from # 130
next 10 from # 140
next 10 from # 150
next 10 from # 160
next 10 from # 170
next 10 from # 180
next 10 from # 190
next 10 from # 200
next 10 from # 210
next 10 from # 220
next 10 from # 230
next 10 from # 240
next 10 from # 250
next 10 from # 260
next 10 from # 270
next 10 from # 280
next 10 from # 290
next 10 from # 300
next 10 from # 310
next 10 from # 320
next 10 from # 330
next 10 from # 340
next 10 from # 350
next 10 from # 360
next 10 from # 370
next 10 from # 380
next 10 from # 390
next 10 from # 400
next 10 from # 410
next 10 from # 420
next 10 from # 430
next 10 from # 440
next 10 from # 450
next 10 from # 460
next 10 from # 470
next 10 from # 480
next 10 from # 490
next 10 from # 500
next 10 from # 510
next 10 from # 520
next 10 from # 530
next 10 from # 540
next 10 from # 550
next 10 from # 560
next 10 from # 570
next 10 from # 580
next 10 from # 590
next 10 from # 600
next 10 from # 610
next 10 from

...

next 10 from # 300
next 10 from # 310
next 10 from # 320
next 10 from # 330
next 10 from # 340
next 10 from # 350
next 10 from # 360
next 10 from # 370
next 10 from # 380
next 10 from # 390
next 10 from # 400
next 10 from # 410
next 10 from # 420
next 10 from # 430
next 10 from # 440
next 10 from # 450
next 10 from # 460
https://www.indeed.com/jobs?q=data+science&l=Bethesda%2C+MD&radius=10
802
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
next 10 from # 50
next 10 from # 60
next 10 from # 70
next 10 from # 80
next 10 from # 90
next 10 from # 100
next 10 from # 110
next 10 from # 120
next 10 from # 130
next 10 from # 140
next 10 from # 150
next 10 from # 160
next 10 from # 170
next 10 from # 180
next 10 from # 190
next 10 from # 200
next 10 from # 210
next 10 from # 220
next 10 from # 230
next 10 from # 240
next 10 from # 250
next 10 from # 260
next 10 from # 270
next 10 from # 280
next 10 from # 290
next 10 from # 300
next 10 from # 310
next 10 from # 320
next

next 10 from # 30
next 10 from # 40
next 10 from # 50
https://www.indeed.com/jobs?q=data+science&l=Clifton%2C+NJ&radius=10
45
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
https://www.indeed.com/jobs?q=data+science&l=Newark%2C+NJ&radius=10
1000
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
next 10 from # 50
next 10 from # 60
next 10 from # 70
next 10 from # 80
next 10 from # 90
next 10 from # 100
next 10 from # 110
next 10 from # 120
next 10 from # 130
next 10 from # 140
next 10 from # 150
next 10 from # 160
next 10 from # 170
next 10 from # 180
next 10 from # 190
next 10 from # 200
next 10 from # 210
next 10 from # 220
next 10 from # 230
next 10 from # 240
next 10 from # 250
next 10 from # 260
next 10 from # 270
next 10 from # 280
next 10 from # 290
next 10 from # 300
next 10 from # 310
next 10 from # 320
next 10 from # 330
next 10 from # 340
next 10 from # 350
next 10 from # 360
next 10 from # 370
next 10 from # 380
next 10 from # 3

265
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
next 10 from # 50
next 10 from # 60
next 10 from # 70
next 10 from # 80
next 10 from # 90
next 10 from # 100
next 10 from # 110
next 10 from # 120
next 10 from # 130
next 10 from # 140
next 10 from # 150
next 10 from # 160
next 10 from # 170
next 10 from # 180
next 10 from # 190
next 10 from # 200
next 10 from # 210
next 10 from # 220
next 10 from # 230
next 10 from # 240
next 10 from # 250
next 10 from # 260
https://www.indeed.com/jobs?q=data+science&l=Union%2C+NJ&radius=10
79
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
next 10 from # 50
next 10 from # 60
next 10 from # 70
https://www.indeed.com/jobs?q=data+science&l=Piscataway%2C+NJ&radius=10
32
next 10 from # 10
next 10 from # 20
next 10 from # 30
https://www.indeed.com/jobs?q=data+science&l=Paramus%2C+NJ&radius=10
34
next 10 from # 10
next 10 from # 20
next 10 from # 30
https://www.indeed.com/jobs?q=data+science&l=Mount+Laurel%2C

6
https://www.indeed.com/jobs?q=data+science&l=Gorham%2C+ME&radius=10
15
next 10 from # 10
https://www.indeed.com/jobs?q=data+science&l=York%2C+ME&radius=10
5
https://www.indeed.com/jobs?q=data+science&l=Bangor%2C+ME&radius=10
1
https://www.indeed.com/jobs?q=data+science&l=Auburn%2C+ME&radius=10
1
https://www.indeed.com/jobs?q=data+science&l=Saco%2C+ME&radius=10
3
https://www.indeed.com/jobs?q=data+science&l=Kennebunk%2C+ME&radius=10
18
next 10 from # 10
https://www.indeed.com/jobs?q=data+science&l=Presque+Isle%2C+ME&radius=10
https://www.indeed.com/jobs?q=data+science&l=South+Portland%2C+ME&radius=10
17
next 10 from # 10
https://www.indeed.com/jobs?q=data+science&l=Westbrook%2C+ME&radius=10
17
next 10 from # 10
https://www.indeed.com/jobs?q=data+science&l=Freeport%2C+ME&radius=10
1
https://www.indeed.com/jobs?q=data+science&l=Rockland%2C+ME&radius=10
https://www.indeed.com/jobs?q=data+science&l=Belfast%2C+ME&radius=10
https://www.indeed.com/jobs?q=data+science&l=Lewiston%2C+ME&radius=

53
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
next 10 from # 50
https://www.indeed.com/jobs?q=data+science&l=Plano%2C+TX&radius=10
72
next 10 from # 10
next 10 from # 20
next 10 from # 30
next 10 from # 40
next 10 from # 50
next 10 from # 60
next 10 from # 70
https://www.indeed.com/jobs?q=data+science&l=Lubbock%2C+TX&radius=10
https://www.indeed.com/jobs?q=data+science&l=Round+Rock%2C+TX&radius=10
23
next 10 from # 10
next 10 from # 20
https://www.indeed.com/jobs?q=data+science&l=Sugar+Land%2C+TX&radius=10
13
next 10 from # 10
https://www.indeed.com/jobs?q=data+science&l=The+Woodlands%2C+TX&radius=10
14
next 10 from # 10


In [25]:
import pandas as pd

df = pd.DataFrame(frame)
df.shape

(64287, 7)

In [28]:
df.tail(3)

,company,date,job_description,job_title,job_url,location,salary
64284,Robert Half Technology,22 days ago,Data Engineer x 4\n? Master?s or Bachelor?s de...,Data Engineer,http://www.indeed.com/rc/clk?jk=66959d10a8c506...,"Spring, TX 77389",Contract
64285,Conn's Home Plus,30+ days ago,Conn’s History:\n\nConn’s is a 128-year old co...,Senior Credit Risk Analyst,http://www.indeed.com/rc/clk?jk=9570f498330d1d...,"Spring, TX 77381",
64286,Matrix Resources,13 days ago,,Data Engineer,http://www.indeed.com/rc/clk?jk=82eef152a602a1...,"Spring, TX 77389",


In [20]:
# Since we set up the radius criteria to 10 miles, there are a lot of duplicates in the data. 
# Let's get rid of them and save the final data set to scv file

df = df.drop_duplicates()
df.shape

(26634, 7)

In [26]:
df.to_csv("data_science_jobs_USA", encoding='utf-8', index=False)